In [27]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.layers import Layer
import numpy as np
import tensorflow_hub as hub
import tensorflow_datasets as tfds


sound_vocab = {"<pad>": 0, "cat": 1, "dog": 2, "meow": 3, "bark": 4}
inv_sound_vocab = {v: k for k, v in sound_vocab.items()}
animal_vocab = {"<pad>": 0, "cat": 1, "dog": 2}
inv_animal_vocab = {v: k for k, v in animal_vocab.items()}

dummy_text = "<empty>"
dummy_image = np.zeros((224,224,3), dtype=np.float32)

# Text expert
class HubEmbeddingLayer(Layer):
    def __init__(self, hub_url="https://tfhub.dev/google/universal-sentence-encoder/4"):
        super().__init__()
        self.hub_layer = hub.KerasLayer(hub_url, trainable=True)
    def call(self, inputs):
        return self.hub_layer(inputs)

def build_text_expert():
    inp = Input(shape=(), dtype=tf.string)
    x = HubEmbeddingLayer()(inp)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    out = layers.Dense(len(sound_vocab))(x)
    return Model(inp, out, name="TextExpert")

# Image expert
def build_image_expert():
    base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3), include_top=False,
                                                   weights="imagenet", pooling="avg")
    base_model.trainable = False
    inp = Input(shape=(224,224,3))
    x = base_model(inp)
    x = layers.Dense(128, activation="relu")(x)
    out = layers.Dense(len(animal_vocab))(x)
    return Model(inp, out, name="ImageExpert")

# Deterministic gating layer selecting expert by presence flags
class GateLayer(Layer):
    def call(self, inputs):
        text_present, image_present = inputs  # shape (batch,1)
        gate_text = 1.0 - image_present  # text expert if no image
        gate_image = image_present       # image expert if image present
        gates = tf.concat([gate_text, gate_image], axis=1)
        return gates

def gating_entropy_loss(gate_weights):
    return -tf.reduce_mean(tf.reduce_sum(gate_weights * tf.math.log(gate_weights + 1e-10), axis=1))

class PadLayer(Layer):
    def __init__(self, pad_width):
        super().__init__()
        self.pad_width = pad_width
    def call(self, inputs):
        neg_inf = tf.fill([tf.shape(inputs)[0], self.pad_width], -1e9)
        return tf.concat([inputs, neg_inf], axis=1)

class SoftmaxWithTemp(Layer):
    def __init__(self, temperature=1.0):
        super().__init__()
        self.temperature = temperature
    def call(self, inputs):
        return tf.nn.softmax(inputs / self.temperature)

def load_image_dataset():
    ds, info = tfds.load('cats_vs_dogs', split='train', with_info=True)
    cat_images, dog_images = [], []
    for ex in ds.as_numpy_iterator():
        label = ex['label']
        image = ex['image']
        image = tf.image.resize(image, (224,224)).numpy() / 255.0
        if label == 0 and len(cat_images) < 10:
            cat_images.append(image)
        elif label == 1 and len(dog_images) < 10:
            dog_images.append(image)
        if len(cat_images) == 10 and len(dog_images) == 10:
            break
    images = np.concatenate([np.array(cat_images), np.array(dog_images)], axis=0)
    labels = np.array([animal_vocab['cat']] * 10 + [animal_vocab['dog']] * 10, dtype=np.int32)
    text_inputs = np.array([dummy_text] * len(images))
    text_present = np.zeros((len(images),1), dtype=np.float32)
    image_present = np.ones((len(images),1), dtype=np.float32)
    return tf.data.Dataset.from_tensor_slices(((text_inputs, images, text_present, image_present), labels))

def load_text_dataset():
    texts = ["cat", "dog", "cat", "dog", "cat", "dog", "cat", "dog"]
    sound_labels = np.array(
        [sound_vocab["meow"], sound_vocab["bark"], sound_vocab["meow"], sound_vocab["bark"],
         sound_vocab["meow"], sound_vocab["bark"], sound_vocab["meow"], sound_vocab["bark"]],
        dtype=np.int32)
    text_present = np.ones((len(texts),1), dtype=np.float32)
    image_present = np.zeros((len(texts),1), dtype=np.float32)
    dummy_images = np.zeros((len(texts), 224, 224, 3), dtype=np.float32)
    return tf.data.Dataset.from_tensor_slices(((texts, dummy_images, text_present, image_present), sound_labels))

def prepare_combined_dataset():
    image_ds = load_image_dataset()
    text_ds = load_text_dataset()
    combined = image_ds.concatenate(text_ds)
    combined = combined.shuffle(32).batch(4).prefetch(tf.data.AUTOTUNE)
    return combined


class MoeModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.text_expert = build_text_expert()
        self.image_expert = build_image_expert()
        self.gate = GateLayer()
        self.pad_layer = PadLayer(len(sound_vocab) - len(animal_vocab))
        self.softmax_temp = SoftmaxWithTemp(0.7)
        self.loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        self.lambda_image_aux = 0.8
        self.lambda_entropy = 0.01
        self.train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
        self.val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    def call(self, inputs, training=False):
        text_input, image_input, text_present, image_present = inputs
        gate_weights = self.gate([text_present, image_present])
        text_logits = self.text_expert(text_input, training=training)
        image_logits = self.image_expert(image_input, training=training)
        image_logits_padded = self.pad_layer(image_logits)

        text_logits_norm = self.softmax_temp(text_logits)
        image_logits_norm = self.softmax_temp(image_logits_padded)

        combined_logits = gate_weights[:, 0:1] * text_logits_norm + gate_weights[:, 1:2] * image_logits_norm
        outputs = tf.nn.softmax(combined_logits)
        return outputs, gate_weights, text_logits, image_logits

    def train_step(self, data):
        (text_input, image_input, text_present, image_present), labels = data
        with tf.GradientTape() as tape:
            outputs, gate_weights, text_logits, image_logits = self((text_input, image_input, text_present, image_present), training=True)

            image_present_flat = tf.reshape(image_present, [-1])
            mask_image = tf.equal(image_present_flat, 1.0)
            mask_text = tf.logical_not(mask_image)

            text_labels = tf.boolean_mask(labels, mask_text)
            text_logits_masked = tf.boolean_mask(text_logits, mask_text)
            text_logits_masked = text_logits_masked[:, :len(sound_vocab)]

            image_labels = tf.boolean_mask(labels, mask_image)
            image_logits_masked = tf.boolean_mask(image_logits, mask_image)
            image_logits_masked = image_logits_masked[:, :len(animal_vocab)]

            text_loss = tf.cond(tf.greater(tf.size(text_labels), 0),
                                lambda: self.loss_fn(text_labels, text_logits_masked),
                                lambda: 0.0)

            image_loss = tf.cond(tf.greater(tf.size(image_labels), 0),
                                lambda: self.loss_fn(image_labels, image_logits_masked),
                                lambda: 0.0)

            entropy_loss = gating_entropy_loss(gate_weights)
            total_loss = text_loss + self.lambda_image_aux * image_loss + self.lambda_entropy * entropy_loss

        grads = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        self.train_accuracy.update_state(labels, outputs)
        return {"loss": total_loss, "train_accuracy": self.train_accuracy.result()}

    def test_step(self, data):
        (text_input, image_input, text_present, image_present), labels = data
        outputs, _, _, _ = self((text_input, image_input, text_present, image_present), training=False)
        loss = self.loss_fn(labels, outputs)
        self.val_accuracy.update_state(labels, outputs)
        return {"loss": loss, "val_accuracy": self.val_accuracy.result()}

moe_model = MoeModel()
moe_model.compile(optimizer=tf.keras.optimizers.Adam())

train_dataset = prepare_combined_dataset()

moe_model.fit(train_dataset, epochs=25)

Epoch 1/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.9639 - train_accuracy: 0.6071
Epoch 2/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.6707 - train_accuracy: 0.6786
Epoch 3/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.9289 - train_accuracy: 0.7143
Epoch 4/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.8789 - train_accuracy: 0.8929
Epoch 5/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.8789 - train_accuracy: 0.9286
Epoch 6/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.8789 - train_accuracy: 0.9286
Epoch 7/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.8789 - train_accuracy: 0.9286
Epoch 8/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.8789 - train_accuracy: 0.9286
Epoch 9/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.8789 - train_accuracy: 0.9286
Epoch 10/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.8789 - train_accuracy: 0.9286
Epoch 11/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.8789 - train_accuracy: 0.9286
Epoch 12/25
7/7 ━━━

In [28]:
def predict_with_moe(text_input=None, image_input=None):
    if text_input is None or text_input == "":
        text_input = dummy_text
    if image_input is None:
        image_input = dummy_image

    if image_input.ndim == 3 and image_input.shape[2] == 1:
        image_input = np.concatenate([image_input]*3, axis=2)

    if image_input.shape != (224,224,3):
        image_input = tf.image.resize(image_input, (224,224)).numpy()

    text_present_f = 0.0 if text_input == dummy_text else 1.0
    image_present_f = 0.0 if np.all(image_input == 0) else 1.0

    preds, gate_weights, text_logits, image_logits = moe_model.predict([
        tf.constant([text_input]),
        np.array([image_input]),
        np.array([[text_present_f]], dtype=np.float32),
        np.array([[image_present_f]], dtype=np.float32)
    ])

    if image_present_f == 1.0:
        image_probs = preds[0][:len(animal_vocab)]
        pred_id = np.argmax(image_probs)
        pred_label = inv_animal_vocab.get(pred_id, "unknown")
    else:
        pred_id = np.argmax(preds[0])
        pred_label = inv_sound_vocab.get(pred_id, "unknown")

    print(f"Softmax output vector: {preds[0]}")
    print(f"Gate weights: {gate_weights[0]}")
    print(f"Input: text='{text_input}', image_present={image_present_f}")
    print(f"Prediction: {pred_label}\n{'-'*50}")
    return pred_label




# Example Tests

print("Test: text input 'dog'")
predict_with_moe(text_input="dog")

print("Test: text input 'canine'")
predict_with_moe(text_input="canine")

print("Test: text input 'cat'")
predict_with_moe(text_input="cat")

print("Test: text input 'feline'")
predict_with_moe(text_input="feline")

from PIL import Image
import requests
import io

# Example real image test (you may replace URL with your own image links)
def load_image(url):
    response = requests.get(url)
    img = Image.open(io.BytesIO(response.content)).convert('RGB')
    img = img.resize((224,224))
    img_array = np.array(img) / 255.0
    return img_array

cat_img_url = "https://images.unsplash.com/photo-1518791841217-8f162f1e1131"
dog_img_url = "https://images.unsplash.com/photo-1517423440428-a5a00ad493e8"

print("Test: image of cat")
predict_with_moe(image_input=load_image(cat_img_url))

print("Test: image of dog")
predict_with_moe(image_input=load_image(dog_img_url))

print("Test: text 'cat' and image cat")
predict_with_moe(text_input="cat", image_input=load_image(cat_img_url))

print("Test: text 'cat' and image dog")
predict_with_moe(text_input="cat", image_input=load_image(dog_img_url))

print("Test: text 'dog' and image dog")
predict_with_moe(text_input="dog", image_input=load_image(dog_img_url))

print("Test: text 'dog' and image cat")
predict_with_moe(text_input="dog", image_input=load_image(cat_img_url))

Test: text input 'dog'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 450ms/step
Softmax output vector: [0.19037393 0.18719237 0.19139642 0.19163263 0.23940468]
Gate weights: [1. 0.]
Input: text='dog', image_present=0.0
Prediction: bark
--------------------------------------------------
Test: text input 'canine'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Softmax output vector: [0.19367616 0.19007787 0.1937225  0.19740176 0.22512168]
Gate weights: [1. 0.]
Input: text='canine', image_present=0.0
Prediction: bark
--------------------------------------------------
Test: text input 'cat'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Softmax output vector: [0.18833591 0.19395396 0.19642694 0.23239923 0.18888393]
Gate weights: [1. 0.]
Input: text='cat', image_present=0.0
Prediction: meow
--------------------------------------------------
Test: text input 'feline'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Softmax output vector: [0.19249156 0.19474043 0.19746724 0.2214343  0.19386642]
Gate weights: [1. 0.]
Input: text='feline', i

'cat'

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, Softmax, LSTM, Conv2D, Flatten, MaxPooling2D, LayerNormalization, Dropout, Input, Conv2DTranspose, Reshape, Add, Lambda, BatchNormalization, add, Activation
from tensorflow.keras import Model
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

In [ ]:
# ---------------------------------------------
# MoE Text Generation Model Components
# ---------------------------------------------

class Expert(Model):
    def __init__(self, d_model):
        super().__init__()
        self.norm = LayerNormalization()
        self.dense1 = Dense(64, activation='relu')
        self.dropout = Dropout(0.2)
        self.dense2 = Dense(d_model)
    def call(self, x):
        x = self.norm(x)
        x = self.dense1(x)
        x = self.dropout(x)
        return self.dense2(x)

class GatingNetwork(Model):
    def __init__(self, d_model, num_experts):
        super().__init__()
        self.dense = Dense(num_experts)
        self.softmax = Softmax(axis=-1)
    def call(self, x):
        logits = self.dense(x)
        return self.softmax(logits)

class MoEResponseGenerator(Model):
    def __init__(self, vocab_size, d_model, num_experts, max_resp_len, lstm_units=128):
        super().__init__()
        self.embedding = Embedding(vocab_size, d_model, mask_zero=True)
        self.global_pool = GlobalAveragePooling1D()
        self.num_experts = num_experts
        self.experts = [Expert(d_model) for _ in range(num_experts)]
        self.gating_net = GatingNetwork(d_model, num_experts)
        self.lstm_units = lstm_units
        self.lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
        self.to_h = Dense(lstm_units)
        self.to_c = Dense(lstm_units)
        self.output_layer = Dense(vocab_size)
        self.max_resp_len = max_resp_len

    def call(self, inputs, training=False):
        input_seq, resp_in_seq = inputs
        enc_emb = self.embedding(input_seq)
        pooled = self.global_pool(enc_emb)

        gating_probs = self.gating_net(pooled)
        expert_outs = tf.stack([expert(pooled) for expert in self.experts], axis=1)
        gated_rep = tf.reduce_sum(tf.expand_dims(gating_probs, 2) * expert_outs, axis=1)

        h_state = self.to_h(gated_rep)
        c_state = self.to_c(gated_rep)

        resp_emb = self.embedding(resp_in_seq)
        lstm_out, _, _ = self.lstm(resp_emb, initial_state=[h_state, c_state])

        logits = self.output_layer(lstm_out)
        return logits, gating_probs

In [ ]:
# ---------------------------------------------
# Utility functions for text encoding and generation
# ---------------------------------------------

# (Your train_data list is assumed defined here - omitted for brevity, but same as your list)

# Build vocabulary based on train_data (input + responses)
train_data = [
    ("hello there", "hi, how can I help"),
    ("hi", "hello, what can I do"),
    ("goodbye", "goodbye, have a nice day"),
    ("see you later", "see you soon, goodbye"),
    ("I want to order pizza", "sure, what toppings do you want"),
    ("can I get a burger", "what size burger would you like"),
    ("what is the weather", "the weather today is sunny"),
    ("is it raining", "no rain expected today"),
    ("hey, I want some pasta", "what kind of pasta would you prefer"),
    ("do you have vegetarian options?", "yes, we have several vegetarian dishes"),
    ("good morning", "good morning, how may I assist you?"),
    ("bye", "take care, see you later"),
    ("will it be hot today?", "expect warm temperatures all day"),
    ("can I order a salad?", "what dressing would you like on your salad?"),
    ("thanks, goodbye", "you're welcome, goodbye!"),
    ("tell me the forecast", "the forecast shows clear skies"),
    ("what's your name?", "i am your assistant, here to help"),
    ("can I have a coffee?", "sure, would you like it black or with milk?"),
    ("thank you for the help", "happy to assist you anytime"),
    ("are you open today?", "yes, we are open from 9 am to 9 pm"),
    ("could you help me with my order", "of course, what would you like to order"),
    ("are there any gluten free options", "yes, we have several gluten free dishes available"),
    ("what are today's specials", "today's special is grilled salmon with vegetables"),
    ("how late are you open", "we are open until 10 pm tonight"),
    ("can you recommend a dessert", "our chocolate lava cake is very popular"),
    ("I need to change my order", "sure, what changes would you like to make"),
    ("do you deliver", "yes, we deliver within a 5 mile radius"),
    ("what payment methods do you accept", "we accept cash, credit cards, and mobile payments"),
    ("is there a parking facility", "yes, free parking is available behind the restaurant"),
    ("thank you very much", "you're welcome, happy to help"),
    ("I have a food allergy", "please let us know your allergy, and we will accommodate"),
    ("can I book a table", "yes, for how many people and what time"),
    ("what's your restaurant address", "we are located at 123 Main Street"),
    ("do you have vegan meals", "yes, we offer delicious vegan options"),
    ("can I get nutritional information", "nutritional info is available on our website"),
    ("how long is the wait time", "usually about 15 minutes during peak hours"),
    ("do you have a kids menu", "yes, we have a special menu for children"),
    ("can I cancel my order", "please provide your order number to cancel"),
    ("what are your opening hours", "we are open from 9 am to 10 pm daily"),
    ("is takeout available", "yes, you can order takeout anytime during opening hours"),        
]

all_texts = [t[0] + " " + t[1] for t in train_data]
all_words = set(word for sentence in all_texts for word in sentence.lower().split())
word2idx = {w: i + 1 for i, w in enumerate(sorted(all_words))}
idx2word = np.array(['<pad>'] + sorted(all_words))
vocab_size = len(idx2word)
max_input_len = 6
max_resp_len = 8

def encode_sentence(sent, max_len):
    words = sent.lower().split()
    seq = [word2idx.get(w, 0) for w in words]
    seq = seq[:max_len] + [0] * (max_len - len(seq))
    return seq

def sample_from_logits(logits, temperature=1.0, top_k=5):
    logits = logits / temperature
    if top_k > 0:
        values, _ = tf.math.top_k(logits, k=top_k)
        min_values = values[:, -1, None]
        logits = tf.where(
            logits < min_values,
            tf.fill(tf.shape(logits), float('-inf')),
            logits,
        )
    probabilities = tf.nn.softmax(logits)
    next_token = tf.random.categorical(tf.math.log(probabilities), num_samples=1)
    return tf.squeeze(next_token, axis=-1).numpy()

def generate_response(model, input_text, max_len=30, temperature=1.0, top_k=5):
    input_seq = np.array([encode_sentence(input_text, max_input_len)])
    response_seq = np.zeros((1, max_resp_len), dtype=np.int32)
    generated_tokens = []
    gating_probs = None

    for i in range(max_len):
        logits, gating_probs = model((input_seq, response_seq), training=False)
        logits_step = logits[:, i % max_resp_len, :]
        next_token = sample_from_logits(logits_step, temperature=temperature, top_k=top_k)[0]
        if next_token == 0:
            break
        generated_tokens.append(idx2word[next_token])
        if i + 1 < max_resp_len:
            response_seq[0, i] = next_token
        else:
            response_seq = np.roll(response_seq, -1, axis=1)
            response_seq[0, -1] = next_token

    top_expert = np.argmax(gating_probs[0].numpy()) if gating_probs is not None else -1
    return " ".join(generated_tokens), top_expert, gating_probs[0].numpy()

# Build training dataset
X_input = np.array([encode_sentence(t[0], max_input_len) for t in train_data])
X_resp_in = np.array([encode_sentence(t[1], max_resp_len) for t in train_data])
X_resp_out = np.array([encode_sentence(t[1], max_resp_len)[1:] + [0] for t in train_data])

In [ ]:
# ---------------------------------------------
# MNIST Digit Classifier Components
# ---------------------------------------------

class DigitClassifier(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2D(32, (3,3), activation='relu')
        self.pool1 = MaxPooling2D((2,2))
        self.conv2 = Conv2D(64, (3,3), activation='relu')
        self.pool2 = MaxPooling2D((2,2))
        self.flatten = Flatten()
        self.dense = Dense(64, activation='relu')
        self.out = Dense(10)  # digits 0-9 logits

    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense(x)
        return self.out(x)

def prepare_mnist_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    return (x_train, y_train), (x_test, y_test)

In [ ]:
# ---------------------------------------------
# Training setup and training loops
# ---------------------------------------------

digit_classifier = DigitClassifier()
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = prepare_mnist_data()
digit_classifier.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
print("Training MNIST digit classifier...")
digit_classifier.fit(x_train_mnist, y_train_mnist, epochs=3, batch_size=128, validation_split=0.1)

d_model = 32
num_experts = 5
lstm_units = 128
max_epochs = 300
batch_size = 2

moe_model = MoEResponseGenerator(vocab_size, d_model, num_experts, max_resp_len, lstm_units)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

train_dataset = tf.data.Dataset.from_tensor_slices(((X_input, X_resp_in), X_resp_out)).shuffle(50).batch(batch_size)

@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        logits, _ = moe_model(inputs, training=True)
        loss = loss_fn(labels, logits)
    grads = tape.gradient(loss, moe_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, moe_model.trainable_variables))
    return loss

print("Training MoE text generation model...")
for epoch in range(max_epochs):
    total_loss = 0.0
    for batch in train_dataset:
        loss = train_step(batch[0], batch[1])
        total_loss += loss.numpy()
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch + 1}/{max_epochs}: Loss = {total_loss / len(train_dataset):.4f}")

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

# Load and normalize MNIST data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Prepare target images for increment digits
train_target_images = np.zeros_like(x_train)
for idx in range(len(x_train)):
    inc_digit = (y_train[idx] + 1) % 10
    candidates = np.where(y_train == inc_digit)[0]
    chosen_idx = np.random.choice(candidates)
    train_target_images[idx] = x_train[chosen_idx]

test_target_images = np.zeros_like(x_test)
for idx in range(len(x_test)):
    inc_digit = (y_test[idx] + 1) % 10
    candidates = np.where(y_test == inc_digit)[0]
    chosen_idx = np.random.choice(candidates)
    test_target_images[idx] = x_test[chosen_idx]

# Residual block
def residual_block(x, filters, kernel_size=3):
    shortcut = x
    x = Conv2D(filters, kernel_size, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(filters, kernel_size, padding="same")(x)
    x = BatchNormalization()(x)
    x = add([shortcut, x])
    x = Activation("relu")(x)
    return x

# Build model with residual blocks
def build_improved_increment_model():
    inputs = Input(shape=(28,28,1))
    x = Conv2D(64, 3, strides=2, padding="same", activation="relu")(inputs)
    x = BatchNormalization()(x)
    x = residual_block(x, 64)
    x = Conv2D(128, 3, strides=2, padding="same", activation="relu")(x)
    x = BatchNormalization()(x)
    x = residual_block(x, 128)

    x = Flatten()(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.3)(x)

    x = Dense(7*7*128, activation="relu")(x)
    x = Reshape((7,7,128))(x)
    x = Conv2DTranspose(128, 3, strides=2, padding="same", activation="relu")(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(64, 3, strides=2, padding="same", activation="relu")(x)
    x = BatchNormalization()(x)

    outputs = Conv2D(1, 3, padding="same", activation="sigmoid")(x)
    model = Model(inputs, outputs)
    return model

increment_model = build_improved_increment_model()

# Initialize VGG16 model for perceptual loss - with input shape 32x32x3 (minimum required)
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
vgg.trainable = False
feature_extractor = Model(vgg.input, vgg.get_layer("block3_conv3").output)

# Preprocessing function to resize and replicate grayscale channel for VGG16
def preprocess_for_vgg(x):
    x_resized = tf.image.resize(x, [32, 32])
    x_rgb = tf.image.grayscale_to_rgb(x_resized)
    x_preprocessed = preprocess_input(x_rgb * 255.0)
    return x_preprocessed

# Perceptual loss combining MSE and VGG feature loss
def perceptual_loss(y_true, y_pred):
    y_true_pp = preprocess_for_vgg(y_true)
    y_pred_pp = preprocess_for_vgg(y_pred)
    f_true = feature_extractor(y_true_pp)
    f_pred = feature_extractor(y_pred_pp)
    return tf.reduce_mean(tf.square(f_true - f_pred))

def combined_loss(y_true, y_pred):
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    pl = perceptual_loss(y_true, y_pred)
    return mse + 0.1 * pl

increment_model.compile(optimizer='adam', loss=combined_loss)

# Train with learning rate scheduler callback
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3, verbose=1)

increment_model.fit(
    x_train,
    train_target_images,
    epochs=30,
    batch_size=128,
    validation_data=(x_test, test_target_images),
    callbacks=[lr_scheduler]
)

# Inference helper
def get_incremented_digit_image(input_image):
    img = input_image.astype("float32") / 255.0
    if img.ndim == 2:
        img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)
    generated_img = increment_model.predict(img)
    return generated_img[0]

In [ ]:
# Visualization of test results for a random sample
# Visualization test
idx = np.random.randint(len(x_test))
input_img = x_test[idx]
original_digit = y_test[idx]
inc_digit = (original_digit + 1) % 10
generated_img = get_incremented_digit_image(input_img)

plt.figure(figsize=(12, 4))
plt.subplot(1,3,1)
plt.title(f"Input: {original_digit}")
plt.imshow(input_img[:,:,0], cmap='gray')
plt.axis('off')

plt.subplot(1,3,2)
plt.title(f"Generated: {inc_digit}")
plt.imshow(generated_img[:,:,0], cmap='gray')
plt.axis('off')

plt.subplot(1,3,3)
plt.title(f"Expected: {inc_digit}")
expected_img = x_test[np.where(y_test == inc_digit)[0][0]]
plt.imshow(expected_img[:,:,0], cmap='gray')
plt.axis('off')

plt.show()


In [ ]:
def multimodal_handler(input_text=None, input_image=None):
    if input_text:
        response, expert, gating_probs = generate_response(moe_model, input_text)
        print(f"Text Input: {input_text}")
        print(f"Response: {response}")
        print(f"Top expert used: {expert}")
        print(f"Gating probs: {np.round(gating_probs, 3)}")
    if input_image is not None:
        pred_digit, inc_digit, inc_image = get_incremented_digit_image_generative(input_image)
        print(f"Image input digit: {pred_digit}, incremented output digit: {inc_digit}")
        plt.subplot(1, 2, 1)
        plt.imshow(input_image, cmap='gray')
        plt.title(f"Input: {pred_digit}")
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.imshow(inc_image, cmap='gray')
        plt.title(f"Output: {inc_digit}")
        plt.axis('off')
        plt.show()


In [ ]:
# ---------------------------------------------
# Demo code to test multimodal system
# ---------------------------------------------

# Text only
multimodal_handler(input_text="hello, how are you?")

# Image only (from MNIST test)
multimodal_handler(input_image=x_test_mnist[15])

# Both
multimodal_handler(input_text="What number is this?", input_image=x_test_mnist[42])

In [ ]:
greeting_examples = [
    "hello",
    "hi there",
    "good morning",
    "hey, how are you?",
    "what's up?"
]

goodbye_examples = [
    "goodbye",
    "see you later",
    "bye for now",
    "talk to you soon",
    "have a great day"
]

order_food_examples = [
    "I want to order a pizza",
    "can I get a burger please?",
    "what sides do you have?",
    "I'd like a vegetarian pasta",
    "do you have gluten free options?"
]

weather_examples = [
    "what's the weather today?",
    "will it rain tomorrow?",
    "is it sunny outside?",
    "what's the forecast for this week?",
    "do I need an umbrella today?"
]

miscellaneous_examples = [
    "what's your name?",
    "can you tell me a joke?",
    "how do I say goodbye politely?",
    "are you open on weekends?",
    "what time do you close?"
]

inference_prompts = greeting_examples + goodbye_examples + order_food_examples + weather_examples + miscellaneous_examples
for input_text in inference_prompts:
    multimodal_handler(input_text)

    # print("Input:", input_text)
    # print("Generated response:", response)
    # print("Top expert used:", expert_used)
    # print("Gating probabilities:", np.round(gating_distribution, 3))
    print("---------------------------------")

# Old Code

In [ ]:
# Sample data (input, response)
train_data = [
    ("hello there", "hi, how can I help"),
    ("hi", "hello, what can I do"),
    ("goodbye", "goodbye, have a nice day"),
    ("see you later", "see you soon, goodbye"),
    ("I want to order pizza", "sure, what toppings do you want"),
    ("can I get a burger", "what size burger would you like"),
    ("what is the weather", "the weather today is sunny"),
    ("is it raining", "no rain expected today"),
    ("hey, I want some pasta", "what kind of pasta would you prefer"),
    ("do you have vegetarian options?", "yes, we have several vegetarian dishes"),
    ("good morning", "good morning, how may I assist you?"),
    ("bye", "take care, see you later"),
    ("will it be hot today?", "expect warm temperatures all day"),
    ("can I order a salad?", "what dressing would you like on your salad?"),
    ("thanks, goodbye", "you're welcome, goodbye!"),
    ("tell me the forecast", "the forecast shows clear skies"),
    ("what's your name?", "i am your assistant, here to help"),
    ("can I have a coffee?", "sure, would you like it black or with milk?"),
    ("thank you for the help", "happy to assist you anytime"),
    ("are you open today?", "yes, we are open from 9 am to 9 pm"),
    ("could you help me with my order", "of course, what would you like to order"),
    ("are there any gluten free options", "yes, we have several gluten free dishes available"),
    ("what are today's specials", "today's special is grilled salmon with vegetables"),
    ("how late are you open", "we are open until 10 pm tonight"),
    ("can you recommend a dessert", "our chocolate lava cake is very popular"),
    ("I need to change my order", "sure, what changes would you like to make"),
    ("do you deliver", "yes, we deliver within a 5 mile radius"),
    ("what payment methods do you accept", "we accept cash, credit cards, and mobile payments"),
    ("is there a parking facility", "yes, free parking is available behind the restaurant"),
    ("thank you very much", "you're welcome, happy to help"),
    ("I have a food allergy", "please let us know your allergy, and we will accommodate"),
    ("can I book a table", "yes, for how many people and what time"),
    ("what's your restaurant address", "we are located at 123 Main Street"),
    ("do you have vegan meals", "yes, we offer delicious vegan options"),
    ("can I get nutritional information", "nutritional info is available on our website"),
    ("how long is the wait time", "usually about 15 minutes during peak hours"),
    ("do you have a kids menu", "yes, we have a special menu for children"),
    ("can I cancel my order", "please provide your order number to cancel"),
    ("what are your opening hours", "we are open from 9 am to 10 pm daily"),
    ("is takeout available", "yes, you can order takeout anytime during opening hours"),        
]

# Build vocabulary
all_texts = [t[0] + " " + t[1] for t in train_data]
all_words = set(word for sentence in all_texts for word in sentence.lower().split())
word2idx = {w: i + 1 for i, w in enumerate(sorted(all_words))}
idx2word = np.array(['<pad>'] + sorted(all_words))
vocab_size = len(idx2word)

max_input_len = 6
max_resp_len = 8

def encode_sentence(sent, max_len):
    words = sent.lower().split()
    seq = [word2idx.get(w, 0) for w in words]
    seq = seq[:max_len] + [0] * (max_len - len(seq))
    return seq

X_input = np.array([encode_sentence(t[0], max_input_len) for t in train_data])
X_resp_in = np.array([encode_sentence(t[1], max_resp_len) for t in train_data])
X_resp_out = np.array([encode_sentence(t[1], max_resp_len)[1:] + [0] for t in train_data]) # shifted

class Expert(Model):
    def __init__(self, d_model):
        super().__init__()
        self.norm = tf.keras.layers.LayerNormalization()
        self.dense1 = Dense(64, activation='relu')
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.dense2 = Dense(d_model)
    def call(self, x):
        x = self.norm(x)
        x = self.dense1(x)
        x = self.dropout(x)
        return self.dense2(x)

class GatingNetwork(Model):
    def __init__(self, d_model, num_experts):
        super().__init__()
        self.dense = Dense(num_experts)
        self.softmax = Softmax(axis=-1)
    def call(self, x):
        logits = self.dense(x)
        return self.softmax(logits)

class MoEResponseGenerator(Model):
    def __init__(self, vocab_size, d_model, num_experts, max_resp_len, lstm_units=128):
        super().__init__()
        self.embedding = Embedding(vocab_size, d_model, mask_zero=True)
        self.global_pool = GlobalAveragePooling1D()
        self.num_experts = num_experts
        self.experts = [Expert(d_model) for _ in range(num_experts)]
        self.gating_net = GatingNetwork(d_model, num_experts)
        self.lstm_units = lstm_units
        self.lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
        self.to_h = Dense(lstm_units)
        self.to_c = Dense(lstm_units)
        self.output_layer = Dense(vocab_size)
        self.max_resp_len = max_resp_len

    def call(self, inputs, training=False):
        input_seq, resp_in_seq = inputs
        enc_emb = self.embedding(input_seq)
        pooled = self.global_pool(enc_emb)

        gating_probs = self.gating_net(pooled)
        expert_outs = tf.stack([expert(pooled) for expert in self.experts], axis=1)
        gated_rep = tf.reduce_sum(tf.expand_dims(gating_probs, 2) * expert_outs, axis=1)

        h_state = self.to_h(gated_rep)
        c_state = self.to_c(gated_rep)

        resp_emb = self.embedding(resp_in_seq)
        lstm_out, _, _ = self.lstm(resp_emb, initial_state=[h_state, c_state])

        logits = self.output_layer(lstm_out)
        return logits, gating_probs

# Hyperparams and dataset
d_model = 32
num_experts = 4
batch_size = 2
epochs = 250

dataset = tf.data.Dataset.from_tensor_slices(((X_input, X_resp_in), X_resp_out)).shuffle(20).batch(batch_size)

model = MoEResponseGenerator(vocab_size, d_model, num_experts, max_resp_len)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        logits, _ = model(inputs, training=True)
        loss = loss_fn(labels, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

for epoch in range(epochs):
    total_loss = 0
    for in_batch, out_batch in dataset:
        loss = train_step(in_batch, out_batch)
        total_loss += loss.numpy()
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}/{epochs} Loss: {total_loss/len(dataset):.4f}")

In [ ]:
def sample_from_logits(logits, temperature=1.0, top_k=5):
    logits = logits / temperature
    if top_k > 0:
        values, _ = tf.math.top_k(logits, k=top_k)
        min_values = values[:, -1, None]
        logits = tf.where(
            logits < min_values,
            tf.fill(tf.shape(logits), float('-inf')),
            logits,
        )
    probabilities = tf.nn.softmax(logits)
    next_token = tf.random.categorical(tf.math.log(probabilities), num_samples=1)
    return tf.squeeze(next_token, axis=-1).numpy()

def generate_response(model, input_text, max_len=30, temperature=1.0, top_k=5):
    input_seq = np.array([encode_sentence(input_text, max_input_len)])
    response_seq = np.zeros((1, max_resp_len), dtype=np.int32)
    generated_tokens = []
    gating_probs = None

    for i in range(max_len):
        logits, gating_probs = model((input_seq, response_seq), training=False)
        logits_step = logits[:, i % max_resp_len, :]
        
        next_token = sample_from_logits(logits_step, temperature=temperature, top_k=top_k)[0]

        if next_token == 0:  # end on padding token
            break
        
        generated_tokens.append(idx2word[next_token])
        if i + 1 < max_resp_len:
            response_seq[0, i] = next_token
        else:
            response_seq = np.roll(response_seq, -1, axis=1)
            response_seq[0, -1] = next_token

    top_expert = np.argmax(gating_probs[0].numpy()) if gating_probs is not None else -1
    return " ".join(generated_tokens), top_expert, gating_probs[0].numpy()


# Demo
inference_prompts = [
    "hello there",
    "I want to order pizza",
    "goodbye",
    "can you help me order food",
    "hi, what's going on?",
    "will it rain tomorrow?",
    "how do I say goodbye politely?",
    "what toppings do you have?",
    "is today sunny or cloudy?",
    "see you soon"
]
for input_text in inference_prompts:
    response, expert_used, gating_distribution = generate_response(model, input_text, max_len=500)

    print("Input:", input_text)
    print("Generated response:", response)
    print("Top expert used:", expert_used)
    print("Gating probabilities:", np.round(gating_distribution, 3))
    print("---------------------------------")

In [ ]:
from PIL import Image
import numpy as np

def classify_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image_array = np.array(image)
    mean_colors = image_array.mean(axis=(0, 1))
    color_names = ['red', 'green', 'blue']
    dominant_color = color_names[np.argmax(mean_colors)]
    return f"Dominant color is {dominant_color}."

def multimodal_response(model, input_text=None, image_path=None):
    result = ""
    if input_text:
        # Use your generate_response logic for text
        response, exp_used, gating_distribution = generate_response(model, input_text, max_len=30)
        result += f"Text response: {response}\nTop expert used: {exp_used}\n"
    if image_path:
        # Classify the image
        image_result = classify_image(image_path)
        result += f"Image analysis: {image_result}\n"
    return result

# Example usage:
print(multimodal_response(model, input_text="what toppings do you have?", image_path="sample_image.jpg"))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, Softmax, LSTM, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras import Model
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

# ----------- MNIST DIGIT CLASSIFIER -----------

class DigitClassifier(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2D(32, (3,3), activation='relu')
        self.pool1 = MaxPooling2D((2,2))
        self.conv2 = Conv2D(64, (3,3), activation='relu')
        self.pool2 = MaxPooling2D((2,2))
        self.flatten = Flatten()
        self.dense = Dense(64, activation='relu')
        self.out = Dense(10)  # digits 0-9 logits

    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense(x)
        return self.out(x)

def prepare_mnist_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    return (x_train, y_train), (x_test, y_test)

# Train or load a pretrained digit classifier
digit_classifier = DigitClassifier()
(x_train, y_train), (x_test, y_test) = prepare_mnist_data()
digit_classifier.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
print("Training digit classifier (this may take a bit)...")
digit_classifier.fit(x_train, y_train, epochs=3, batch_size=128, validation_split=0.1)

# ----------- YOUR MOE TEXT GENERATION MODEL (copy your existing) -----------

# (Paste your current MoEResponseGenerator, Expert, GatingNetwork, encode_sentence, sample_from_logits, etc. here)
# For brevity, let's assume it's loaded as `moe_model`

# For this example, just a placeholder generate_response:
def generate_response(model, input_text, max_len=30, temperature=1.0, top_k=5):
    # Use your real model's generation logic here
    return f"Simulated response to: {input_text}", 1, np.array([0.1, 0.7, 0.1, 0.1])

# ----------- MNIST INCREMENT LOGIC -----------

def get_incremented_digit_image(input_image, x_dataset, y_dataset):
    # Normalize pixel values and ensure float32
    img = input_image.astype('float32')
    if img.max() > 1.0:
        img /= 255.0

    # Add channel dimension if missing
    if len(img.shape) == 2:   # grayscale image shape (28,28)
        img = np.expand_dims(img, axis=-1)  # become (28,28,1)

    # Add batch dimension for model input
    img = np.expand_dims(img, axis=0)  # become (1,28,28,1)

    logits = digit_classifier(img)
    pred_digit = tf.argmax(logits, axis=1).numpy()[0]

    inc_digit = 0 if pred_digit == 9 else pred_digit + 1
    idx = np.where(y_dataset == inc_digit)[0][0]
    inc_image = x_dataset[idx][:, :, 0]  # remove channel for display

    return pred_digit, inc_digit, inc_image


# ----------- MULTIMODAL HANDLER -----------

def multimodal_handler(input_text=None, input_image=None):
    if input_text:
        response, expert, gating = generate_response(None, input_text)
        print(f"Text Input: {input_text}")
        print(f"Response: {response}")
        print(f"Top expert used: {expert}")
        print(f"Gating probabilities: {gating}")
    if input_image is not None:
        pred_digit, inc_digit, inc_image = get_incremented_digit_image(input_image, x_test, np.argmax(y_test, axis=1))
        print(f"Input Digit: {pred_digit}, Incremented Digit: {inc_digit}")
        plt.subplot(1,2,1)
        plt.imshow(input_image, cmap='gray')
        plt.title(f"Input: {pred_digit}")
        plt.axis('off')
        plt.subplot(1,2,2)
        plt.imshow(inc_image, cmap='gray')
        plt.title(f"Output: {inc_digit}")
        plt.axis('off')
        plt.show()

# ----------- DEMO -----------

# Text-only
multimodal_handler(input_text="I need help with my order.")

# Image-only
multimodal_handler(input_image=x_test[5])

# Both modalities
multimodal_handler(input_text="What digit is this?", input_image=x_test[7])
